In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime

# Indicator Creation

In this notebook we will create new interesting features which we believe to be relevant for clustering purposes. The indicators we will try to create in this notebook are:
1. Account age in days (From account creation till the release of this dataset) (datetime(2022,9,29,11,0,0))
2. Account average tweets per day
3. Account highest daily tweet count
4. Account average twitter text length
5. Account tweet hashtag average
6. Account discussion creation score (Tweet to retweet ratio)

In [23]:
merged_df = pd.read_csv("dataset/merged_dataset.csv")
merged_df.head(1)

,Unnamed: 0,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text
0,0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu..."


In [17]:
merged_df.axes
merged_df.drop(columns="Unnamed: 0", inplace=True) # Dropping the first column of the dataset, consisting of an index copy
merged_df.axes

[RangeIndex(start=0, stop=10599305, step=1),
 Index(['Unnamed: 0', 'user_id', 'name', 'lang', 'bot', 'account_created',
        'statuses_count', 'tweet_id', 'retweet_count', 'reply_count',
        'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions',
        'tweet_created', 'text'],
       dtype='object')]

[RangeIndex(start=0, stop=10599305, step=1),
 Index(['user_id', 'name', 'lang', 'bot', 'account_created', 'statuses_count',
        'tweet_id', 'retweet_count', 'reply_count', 'favorite_count',
        'num_hashtags', 'num_urls', 'num_mentions', 'tweet_created', 'text'],
       dtype='object')]

## 1. Account age in days
Creating the account age in days feature and adding it to the dataframe.

In [12]:
# Not finished. Need to find a way to add a new column to a dataframe and then iterate values to that column. - Tengel
def calculate_account_age_in_days(created_datetime):
    account_created = created_datetime
    release_of_dataset = datetime(2022,9,29,11,0,0)
    age_of_account = release_of_dataset - account_created
    return age_of_account.days

column_name = "account_age_in_days"

temp_df = merged_df.head(5)
#temp_df = temp_df.concat(axis=1,)
#temp_df['account_age_in_days'] = temp_df[].apply(calculate_account_age_in_days())
temp_df.head()

,Unnamed: 0,user_id,name,lang,bot,created_at_x,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at_y,text
0,0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu..."
1,1,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.795354e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-06-21 08:05:13,"""Happiness is not a station you arrive at, but..."
2,2,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.517009e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-05 12:41:00,Music flow.
3,3,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.919200e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-25 12:17:31,@_SimplyKC follow meeee...
4,4,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.605225e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-29 20:55:05,"""You are the only person on earth who can use ..."
